In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
import numpy as np
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
# init_features, init_labels = np.random.random((100000,100)), np.random.random((100000,1))
graph1 = tf.Graph()
import time

def datagenerator():
    while True:
        time.sleep(1)
        init_features, init_labels = np.random.random((100)), np.random.random((1))
        yield init_features, init_labels

def time_consumer(thresh):
    i = tf.constant(0)
    c = lambda i: tf.less(i, thresh)
    b = lambda i: tf.add(i, 1)
    r = tf.while_loop(c, b, [i], return_same_structure=True)
    return r

with graph1.as_default():
#     input_features = tf.placeholder(tf.float32, shape=[None,100] , name = 'input')
#     input_labels = tf.placeholder(tf.float32, shape=[None, 1], name = 'labels')
#     dataset = tf.data.Dataset.from_tensor_slices((input_features,input_labels))
#     dataset = dataset.batch(100000)
#     dataset = dataset.repeat(-1)
#     dataset = dataset.prefetch(1)
    dataset = tf.data.Dataset().from_generator(datagenerator, output_types= (tf.float32, tf.float32), 
                                           output_shapes=(tf.TensorShape([100]),tf.TensorShape([1]))).batch(10)
#     dataset = dataset.prefetch(10)
    data_iter = dataset.make_initializable_iterator()
    features, labels = data_iter.get_next()
    net = tf.layers.dense(features,100,activation=tf.sigmoid)
    net = tf.layers.dense(net,1,activation=tf.sigmoid)
    loss = tf.losses.mean_squared_error(net,labels)
    with tf.control_dependencies(time_consumer(int(1e7))):
        train_op = tf.train.AdamOptimizer().minimize(loss)
    init = tf.global_variables_initializer()

In [2]:
%%time
with tf.Session(graph=graph1, config = config) as sess:
    sess.run([data_iter.initializer, init])
    for _ in range(2):
        _, loss_val, f = sess.run([train_op, loss, features])
        print(loss_val)

0.09049931
0.103727855
CPU times: user 1min 27s, sys: 613 ms, total: 1min 27s
Wall time: 1min 27s
